# ↔️ Building Dynamic Video Streams with VideoDB: Integrating Custom Data and APIs

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Programmatic_Streams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
Imagine you're watching a captivating keynote session from your favorite conference, and you’re welcomed with a personalized stream just for you.

This tutorial demonstrates how to create dynamic video streams by integrating data from custom databases and external APIs. We'll use a practical example: a recording of a [Config 2023](https://www.youtube.com/watch?v=Nmv8XdFiej0) keynote session. By using VideoDB, we'll show how companies like [Figma](https://www.figma.com/files/recents-and-sharing?fuid=940498258276625180) can personalize the viewing experience for their audience, delivering a richer and more engaging experience.

We'll showcase how to:

* Fetch data from a random user API to represent a hypothetical viewer.
* Integrate this data into a custom VideoDB timeline.
* Create a personalized stream that dynamically displays relevant information alongside the keynote video.

This tutorial is your guide to unlocking the potential of dynamic video streams and transforming your video content with personalized experiences.

## Setup
---


### 📦  Installing packages

In [1]:
%pip -q install videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [2]:
import videodb
import os
from getpass import getpass

# Secure way to enter your VideoDB API key
api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

Please enter your VideoDB API Key: ··········


## Steps
---

### 🔗  Step 1: Connect to VideoDB

Establish a session for uploading videos. Import the necessary modules from VideoDB library to access functionalities.

In [3]:
import videodb
from videodb import connect

conn = videodb.connect()
coll = conn.get_collection()

### 🗳️ Step 2: Upload Base Video
Upload and play the video to ensure it's correctly loaded. We’ll be using [this video](https://www.youtube.com/watch?v=Nmv8XdFiej0) for the purpose of this tutorial.

In [ ]:
# Upload and play a video from a URL
video = coll.upload(url="https://www.youtube.com/watch?v=Nmv8XdFiej0")
print("Uploaded Video ID: ", video.id)

# Alternatively, get a video from your VideoDB collection
# video = coll.get_video('VIDEO_ID_HERE')

Uploaded Video ID:  m-z-019beb55-b24c-7bf3-8160-2615ef3ad396


Lets play the video.

In [ ]:
video.play()

### 📥 Step 3: Fetch Data from a Random User API
This code fetches a random user's data (name and picture) from the "randomuser.me" API.  You can adapt this to retrieve data from any relevant API (e.g., product data, news articles) for your use case.

In [5]:
import requests

# Make a request to the Randomizer API
response = requests.get('https://randomuser.me/api/?results=1&nat=us,ca,gb,au')
data = response.json()

# Extract relevant information
first_name = data['results'][0]['name']['first']
medium_picture = data['results'][0]['picture']['medium']

In [9]:
import requests

# Make a request to the Randomizer API
response = requests.get('https://randomuser.me/api/?results=1&nat=us,ca,gb,au')
data = response.json()

# Extract relevant information
first_name = data['results'][0]['name']['first']
medium_picture = data['results'][0]['picture']['medium']

from IPython.display import Image
print("First Name : ",first_name)
print("Picture URL : ",medium_picture)
Image(url=medium_picture)

First Name :  Debbie
Picture URL :  https://randomuser.me/api/portraits/med/women/65.jpg


### 🚥 Step 4 : Upload the image to VideoDB

- First we download the image to local storage
- Then we use the local path to upload it to VideoDB


In [11]:
import requests

# 1. Download the image locally
local_path = "my_local_image.jpg"

response = requests.get(medium_picture)
if response.status_code == 200:
    with open(local_path, 'wb') as f:
        f.write(response.content)
    print(f"Image downloaded successfully to: {local_path}")
else:
    print(f"Failed to download image. Status code: {response.status_code}")

Image downloaded successfully to: my_local_image.jpg


In [12]:
# 2. Upload using the local file path

from videodb import play_stream, MediaType
image = coll.upload(file_path=local_path, media_type=MediaType.image)

print(f"Image uploaded to VideoDB: {image.id}")

Image uploaded to VideoDB: img-z-019c271b-965a-7142-a97b-0f10c18f4590


### 🧱 Step 5: Create VideoDB Assets

We create VideoDB assets for the base video, the user's name (text), and their picture (image) using the new Editor SDK. The `Font` and `Background` objects allow us to customize the appearance of text elements.

In [28]:
from videodb.editor import (
    Timeline, Track, Clip,
    VideoAsset, TextAsset, ImageAsset,
    Font, Background, Alignment, HorizontalAlignment, VerticalAlignment,
    Position, Offset, Fit
)

# 1. Video Asset (Base background)
video_asset = VideoAsset(id=video.id, start=0)

# 2. Name Asset (Top)
name_asset = TextAsset(
    text=f'Hi {first_name} !',
    font=Font(family="Montserrat", size=60, color="#000000"),
    background=Background(color="#D2C11D", border_width=20, opacity=1.0),
    alignment=Alignment(horizontal=HorizontalAlignment.center, vertical=VerticalAlignment.top),
)

# 3. Message Asset (Middle)
cmon_asset = TextAsset(
    text="Here are your favorite moments",
    font=Font(family="Montserrat", size=60, color="#D2C11D"),
    background=Background(color="#000000", border_width=20, opacity=1.0),
    alignment=Alignment(horizontal=HorizontalAlignment.center, vertical=VerticalAlignment.center),
)

# 4. Image Asset (Bottom)
image_asset = ImageAsset(id=image.id)

### ↔️ Step 6: Create the VideoDB Timeline

Using the `Track` and `Clip` pattern, we arrange and layer assets to create a dynamic video stream. The main video goes on one track, while overlays (name, message, image) go on separate tracks with their start times.

In [35]:
# Create the timeline
timeline = Timeline(conn)

# --- Track 1: Main Video ---
video_track = Track()
video_clip = Clip(asset=video_asset, duration=float(video.length))
video_track.add_clip(0, video_clip)
timeline.add_track(video_track)

# --- Track 2: Overlays ---
overlay_track = Track()

# 1. Add Name Overlay (Top)
name_clip = Clip(
    asset=name_asset,
    duration=4,
    position=Position.top,
    offset=Offset(y=0.15)
)
overlay_track.add_clip(5, name_clip)

# 2. Add Message Overlay (Center)
cmon_clip = Clip(
    asset=cmon_asset,
    duration=4,
    position=Position.center,
)
overlay_track.add_clip(5, cmon_clip)

# 3. Add Image Overlay (Bottom)
image_clip = Clip(

    asset=image_asset,
    duration=4,
    position=Position.bottom,
    scale=2,
    fit=None,
    offset=Offset(y=-0.15)
)
overlay_track.add_clip(5, image_clip)

timeline.add_track(overlay_track)

### ▶️ Step 7: Generate and Play the Personalized Stream

The `generate_stream()` method creates a streamable URL for your personalized video stream. You can then use `play_stream()`  to preview it in your browser.

In [36]:
from videodb import play_stream

stream_url = timeline.generate_stream()
print(stream_url)
play_stream(stream_url)

https://play.videodb.io/v1/af2c71d2-ede1-4ff8-b086-75ad29263e17.m3u8


## Conclusion
---

This tutorial showcased how to create personalized video streams using VideoDB. By integrating data from external APIs and custom databases, you can enhance your video content, personalize user experiences, and unlock new possibilities for engagement. Explore various data sources, experiment with different integrations, and customize your video streams to suit your specific needs.